In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from sklearn.model_selection import KFold
from tensorflow.keras.utils import to_categorical

# Define the create_cnn_model function
def create_cnn_model(input_shape):
    model = Sequential()
    model.add(tf.keras.layers.Input(shape=input_shape))
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(3, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Load dataset
train_df = pd.read_csv('/content/dataset/train.csv')
test_df = pd.read_csv('/content/dataset/test.csv')

# Preprocess labels
label2id = {'Normal/Mild': 0, 'Moderate': 1, 'Severe': 2}
train_df['label'] = train_df['label'].replace(label2id)
train_df['label'] = to_categorical(train_df['label'])

# Create ImageDataGenerator
datagen = ImageDataGenerator(rescale=1.0/255)

# K-Fold Cross Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

input_shape = (128, 128, 1)
fold = 1

for train_index, val_index in kf.split(train_df):
    train_data, val_data = train_df.iloc[train_index], train_df.iloc[val_index]
    
    # Create data generators
    train_generator = datagen.flow_from_dataframe(
        dataframe=train_data,
        directory='/content/dataset/images',
        x_col='filename',
        y_col='label',
        target_size=(128, 128),
        batch_size=32,
        class_mode='categorical',
        color_mode='grayscale' # Use grayscale to match the input shape (128, 128, 1)
    )
    
    val_generator = datagen.flow_from_dataframe(
        dataframe=val_data,
        directory='/content/dataset/images',
        x_col='filename',
        y_col='label',
        target_size=(128, 128),
        batch_size=32,
        class_mode='categorical',
        color_mode='grayscale' # Use grayscale to match the input shape (128, 128, 1)
    )
    
    # Create and train the model
    model = create_cnn_model(input_shape)
    model.fit(train_generator, epochs=10, validation_data=val_generator)

    # Evaluate the model
    val_loss, val_acc = model.evaluate(val_generator)
    print(f'Fold {fold} - Validation Loss: {val_loss}, Validation Accuracy: {val_acc}')
    
    fold += 1

# Test data generator
test_generator = datagen.flow_from_dataframe(
    dataframe=test_df,
    directory='/content/dataset/images',
    x_col='filename',
    y_col=None,
    target_size=(128, 128),
    batch_size=32,
    class_mode=None,
    color_mode='grayscale', # Use grayscale to match the input shape (128, 128, 1)
    shuffle=False
)

# Predict on test data
predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)

# Map predictions back to labels
id2label = {v: k for k, v in label2id.items()}
predicted_labels = [id2label[cls] for cls in predicted_classes]

# Create submission dataframe
submission = pd.DataFrame({
    'filename': test_df['filename'],
    'label': predicted_labels
})

# Save to CSV
submission.to_csv('/content/submission.csv', index=False)